# WB RecSys Project

# Общее описание проекта

Необходимо на основании взаимодействий пользователей с товарами предсказать следующие взаимодействия пользователей с товарами.

# Stage 4

- Выбрать метрику оценки качества и обосновать выбор
- Разработать baseline (может быть несколько алгоритмов)
- Реализовать выбранное решение/я
- Протестировать работу baseline
- Выбрать итоговое решение для дальнейшей оптимизации и обосновать выбор

In [ ]:
import warnings


# ----------------
# Data processing
# ----------------
import dill

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

from tqdm.auto import tqdm


# ---------------------
# RecSys models imports
# ---------------------

from lightgbm import LGBMRanker


# --------------
# Plotting libs
# --------------
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import shap


# -------------------
# Metrics Evaluation
# -------------------
from metrics import RecommenderMetrics


warnings.filterwarnings("ignore")
sns.set_theme(style="whitegrid")

RANDOM_STATE = 42

# Импортируем пути

In [ ]:
data_path = "../data_closed/"

In [ ]:
models_path = "../models/"

In [ ]:
candidates_data_path = models_path + "candidates_data/"

## Transfrorm ITEMS data for RANKER

Немного информации из df_items, а так же преобразуем данную таблицу (закодируем категориальные признаки)

In [ ]:
# Загружаем таблицу айтемов
with open(data_path + "df_items.dill", "rb") as f:
    df_items = dill.load(f)

In [ ]:
df_items.columns

In [ ]:
items_cat_cols = [
    "brand",
    "color",
    "closure",
    "country",
    "cut",
    "height",
    "length",
    "material",
    "model",
    "neckline",
    "pattern",
    "pocket",
    "purpose",
    "sleeve",
]

In [ ]:
items_cat_enc = OrdinalEncoder(dtype=np.int64)
df_items[items_cat_cols] = items_cat_enc.fit_transform(df_items[items_cat_cols])
display(df_items)

In [ ]:
# Save
with open(data_path + "df_items_mod.dill", "wb") as f:
    dill.dump(df_items, f)

# Feature Engineering

In [ ]:
# Загружаем таблицу данных для моделей первого уровня
with open(data_path + "base_models_data.dill", "rb") as f:
    base_models_data = dill.load(f)

# Загружаем таблицу айтемов
with open(data_path + "df_items_mod.dill", "rb") as f:
    df_items = dill.load(f)


In [ ]:
base_models_data = base_models_data.rename(
    columns={
        # переименуем для удобства
        "u_total_inter": "user_hist",
    }
)

# Получаем популярность контента
base_models_data["item_pop"] = base_models_data.groupby("item_id")["user_id"].transform(
    "count"
)
# Получаем среднюю популярность контента, просматриваемого этим юзером
base_models_data["user_avg_pop"] = base_models_data.groupby("user_id")[
    "item_pop"
].transform("mean")

# Получаем среднюю длину истории пользователя, которые смотрит этот контент
base_models_data["item_avg_hist"] = base_models_data.groupby("item_id")[
    "user_hist"
].transform("mean")

# Получаем популярность последнего просмотренного контента
base_models_data.sort_values(
    by=["user_id", "dt"],
    ascending=[True, False],
    ignore_index=True,
    inplace=True,
)
base_models_data["user_last_pop"] = base_models_data.groupby("user_id")[
    "item_pop"
].transform("first")


base_models_data.head(3)

In [ ]:
# Добавляем новые фичи в соответствующие таблицы
df_items = pd.merge(
    left=df_items,
    right=(
        base_models_data[["item_id", "item_pop", "item_avg_hist"]].drop_duplicates()
    ),
    how="left",
    on="item_id",
)

# Создаем таблицу с фитчами пользователей
df_users = base_models_data[
    ["user_id", "user_hist", "user_avg_pop", "user_last_pop"]
].drop_duplicates()

In [ ]:
# Save updated tables

with open(data_path + "df_items_mod.dill", "wb") as f:
    dill.dump(df_items, f)

with open(data_path + "df_users.dill", "wb") as f:
    dill.dump(df_users, f)

## Load data

In [ ]:
# Загружаем таблицу данных для ранкера
with open(data_path + "ranker_data.dill", "rb") as f:
    ranker_data = dill.load(f)


# Загружаем таблицу кандидатов
with open(candidates_data_path + "candidates_full.dill", "rb") as f:
    candidates_full = dill.load(f)

In [ ]:
# Пользователи, которым надо выдавать пресказания для обучения ранкера,
# т.е. присутствуют и в base_models_data и в ranker_data (base to ranker users)
with open(data_path + "b2r_users.dill", "rb") as f:
    b2r_users = dill.load(f)


# Пользователи из test_df, которым будут выданы
# таргетирвонные рекомондации
with open(data_path + "bNr2t_users.dill", "rb") as f:
    bNr2t_users = dill.load(f)


In [ ]:
default_values_candidates = {
    "cos_score": candidates_full["cos_score"].min(),
    "bm25_score": candidates_full["bm25_score"].min(),
    "tfidf_score": candidates_full["tfidf_score"].min(),
    "lfm_score": candidates_full["lfm_score"].min(),
    "cos_rank": candidates_full["cos_rank"].max(),
    "bm25_rank": candidates_full["bm25_rank"].max(),
    "tfidf_rank": candidates_full["tfidf_rank"].max(),
    "lfm_rank": candidates_full["lfm_rank"].max(),
}

# Модель второго уровня (ранкер)

## Ranker Data

### Remove unnecessary

In [ ]:
# Оставим только необходимые параметры из таблицы

# Ранкер будем обучать на пользователях у кого длинная история взаимодействий
ranker_data = ranker_data[ranker_data["u_total_inter"] > 75][
    [
        "user_id",
        "item_id",
        # Так как бьем данные для tain val не по времени,
        # колонка "dt" не нужна
        # --------------------------
        # Потом будем использовать для ранкера чтобы задать таргет
        # (количество взаимодействий с предметом)
        "ui_inter",
        # --------------------------
        # Веса
        "weight",
        "cum_weight",
        # Убираем rel_weight (таскать его нет смысла)
        # на нем обучалась модель первого уровня
        # так что далее он не нужен
        # --------------------------
        # Остальные колонки не нужны
        # Так как они были использованы для вывода весовых колонок,
        # либо присутствуют в фитчах пользователя или айтема
    ]
]

### Train \ Val \ Test Split

In [ ]:
# Теперь ranker_data разбиваем по юзерам
# на train и val для обучения и валидации ранкера
train_size = 0.8
val_size = 0.2


ranker_train_users, ranker_val_users = train_test_split(
    ranker_data[ranker_data["user_id"].isin(b2r_users)]["user_id"],
    random_state=RANDOM_STATE,
    test_size=val_size,
)

# test-выборка у нас уже имеется 
# выборка пользователей присутствующих в base & ranker & test
# на них и будем проводить первичный тест системы
ranker_test_users = bNr2t_users

%clear

In [ ]:
# Оставляем среди users только тех, для кого есть 
# и рекомендации и таргеты
def users_filter(
    user_list: np.ndarray,
    candidates_df: pd.DataFrame,
    df: pd.DataFrame,
) -> pd.DataFrame:
    """
    Filters user interaction data and candidate recommendations, 
    ensuring each user has both interactions and recommendations.

    Args:
        user_list (np.ndarray): User IDs to include.
        candidates_df (pd.DataFrame): Candidate item recommendations 
            with ranks ('cos_rank', 'bm25_rank', 'lfm_rank', 'tfidf_rank').
        df (pd.DataFrame): User-item interactions ('user_id', 'item_id', 'dt', 
            and potentially other weight-based columns).

    Returns:
        pd.DataFrame: Filtered and merged DataFrame with user interactions 
            and candidate items sorted and with missing values filled. 
            It also filters down to items with at least one rank < 15
    """
    # For fillna
    default_values = {
        "ui_inter": 0,
        "weight": 0.0,
        "cum_weight": 0.0,
        **default_values_candidates,
    }

    # Get valid interactions
    df = df[df["user_id"].isin(user_list)]
    candidates_df = candidates_df[candidates_df["user_id"].isin(user_list)]

    # join interaction на наших кандидатов для users из train, val, test
    df = df.merge(
        candidates_df,
        how="outer",
        on=["user_id", "item_id"],
    )

    df.fillna(default_values, inplace=True)
    df["ui_inter"] = df["ui_inter"].astype(int)

    # Сортируем по user_id
    df.sort_values(
        by=["user_id", "item_id"],
        inplace=True,
    )
    
    return df[
        (df["cos_rank"] < 15)
        | (df["bm25_rank"] < 15)
        | (df["lfm_rank"] < 15)
        | (df["tfidf_rank"] < 15)
    ]


In [ ]:
ranker_train = users_filter(ranker_train_users, candidates_full, ranker_data)

# Save 
with open(data_path + "ranker_train.dill", "wb") as f:
    dill.dump(ranker_train, f)

In [ ]:
ranker_train.head(3)

In [ ]:
ranker_val = users_filter(ranker_val_users, candidates_full, ranker_data)

# Save
with open(data_path + "ranker_val.dill", "wb") as f:
    dill.dump(ranker_val, f)

In [ ]:
ranker_val.head(3)

In [ ]:
ranker_test = users_filter(ranker_test_users, candidates_full, ranker_data)

# Save
with open(data_path + "ranker_test.dill", "wb") as f:
    dill.dump(ranker_test, f)

In [ ]:
ranker_test.head(3)

In [ ]:
ranker_train

In [ ]:
ranker_val

## Добавим фитчи предметов и пользователей 

### Пользователей

In [ ]:
# Загружаем таблицу фитчей пользователей
with open(data_path + "df_users.dill", "rb") as f:
    df_users = dill.load(f)

# Для новых фичей юзеров
default_values_users = {
    "user_hist": 0,
    "user_avg_pop": df_users["user_avg_pop"].median(),
    "user_last_pop": df_users["user_last_pop"].median(),
}

In [ ]:
# Добавляем фичи
def add_users_features(
    df: pd.DataFrame,
    users: pd.DataFrame,
) -> pd.DataFrame:
    """
    Merges user and item features into a DataFrame, handling missing values.

    Args:
        df (pd.DataFrame): Interaction DataFrame ('user_id', 'item_id').
        users (pd.DataFrame): User features DataFrame ('user_id').
        items (pd.DataFrame): Item features DataFrame ('item_id').

    Returns:
        pd.DataFrame: DataFrame with merged user and item features, 
            and missing values filled.
    """
    users = users[users["user_id"].isin(df["user_id"])]
    df = pd.merge(df, users, how="left", on=["user_id"])

    # При джойне могут получиться строки
    # с несуществующими айтемами или юзерами.
    # Заполняем пропуски
    df.fillna(default_values_users, inplace=True)

    return df

In [ ]:
# Загрузим таблицу ranker_train
with open(data_path + "ranker_train.dill", "rb") as f:
    ranker_train = dill.load(f) #pl.from_pandas(dill.load(f))

ranker_train = add_users_features(ranker_train, df_users)
# ranker_train = add_items_features(ranker_train, df_items)

# Save 
with open(data_path + "ranker_train.dill", "wb") as f:
    dill.dump(ranker_train, f)

In [ ]:
# Загрузим таблицу ranker_val
with open(data_path + "ranker_val.dill", "rb") as f:
    ranker_val = dill.load(f)

ranker_val = add_users_features(ranker_val, df_users)
# ranker_val = add_items_features(ranker_val, df_users)

# Save
with open(data_path + "ranker_val.dill", "wb") as f:
    dill.dump(ranker_val, f)

In [ ]:
# Загрузим таблицу ranker_test
with open(data_path + "ranker_test.dill", "rb") as f:
    ranker_test = dill.load(f)

ranker_test = add_users_features(ranker_test, df_users)
# ranker_test = add_items_features(ranker_test, df_users)

# Save
with open(data_path + "ranker_test.dill", "wb") as f:
    dill.dump(ranker_test, f)

### Предметов

In [ ]:
# Загружаем таблицу айтемов
with open(data_path + "df_items_mod.dill", "rb") as f:
    df_items = dill.load(f)

# Для новых фичей айтемов
default_values_items = {
    "item_pop": df_items["item_pop"].median(),
    "item_avg_hist": df_items["item_avg_hist"].median(),
}

In [ ]:
# Добавляем фичи
def add_items_features(
    df: pd.DataFrame,
    items: pd.DataFrame,
) -> pd.DataFrame:
    """
    Merges user and item features into a DataFrame, handling missing values.

    Args:
        df (pd.DataFrame): Interaction DataFrame ('user_id', 'item_id').
        items (pd.DataFrame): Item features DataFrame ('item_id').

    Returns:
        pd.DataFrame: DataFrame with merged user and item features,
            and missing values filled.
    """

    items = items[items["item_id"].isin(df["item_id"].unique())]
    df = pd.merge(df, items, how="left", on=["item_id"])

    # # При джойне могут получиться строки
    # # с несуществующими айтемами или юзерами.
    # # Заполняем пропуски
    df.fillna(default_values_items, inplace=True)

    return df

In [ ]:
# Загрузим таблицу ranker_train
with open(data_path + "ranker_train.dill", "rb") as f:
    ranker_train = dill.load(f)


In [ ]:
ranker_train = add_items_features(ranker_train, df_items)

> РЕМАРКА

> Если обучать на большем количестве записей:
Костыли и медж батчами
был вариант слить с помощью Polaris (код есть выше), но сохранить фрейм не получилось --- не хватило памяти на еще одну копию объекта в памяти (питон 🤌)

In [ ]:
# -------
# KOSTYLI
# -------


# # Добавляем фичи
# def add_items_features(
#     df: pl.DataFrame,
#     items: pl.DataFrame,
# ) -> pl.DataFrame:
#     """
#     Merges user and item features into a DataFrame, handling missing values.

#     Args:
#         df (pd.DataFrame): Interaction DataFrame ('user_id', 'item_id').
#         items (pd.DataFrame): Item features DataFrame ('item_id').

#     Returns:
#         pd.DataFrame: DataFrame with merged user and item features,
#             and missing values filled.
#     """
#     df = df.join(
#         items, how="left", on="item_id"
#     )

#     # # При джойне могут получиться строки
#     # # с несуществующими айтемами или юзерами.
#     # # Заполняем пропуски
#     # df = df.to_pandas()
#     # df.fillna(default_values_items, inplace=True)

#     return df


# import gc

# batches = np.array_split(np.array([i for i in ranker_train.index]), 100)

# res_table = []

# for batch in tqdm(batches[75:]):
#     res_table.append(
#         add_items_features(ranker_train[ranker_train.index.isin(batch)], df_items)
#     )
#     gc.collect()


# res_table = pd.concat(
#     res_table,
#     axis=0,
# )


# # Save 
# with open(data_path + "ranker_train_4.dill", "wb") as f:
#     dill.dump(res_table, f)


# # Загрузим таблицу ranker_train
# ranker_train = pd.DataFrame()
# for i in tqdm([1, 2, 3, 4]):
#     with open(data_path + f"ranker_train_{i}.dill", "rb") as f:
#         ranker_train = pd.concat(
#             (
#                 ranker_train,
#                 dill.load(f),
#             ),
#             axis=0,
#         )

In [ ]:
# Save 
with open(data_path + "ranker_train_final.dill", "wb") as f:
    dill.dump(ranker_train, f)

In [ ]:
# Загрузим таблицу ranker_val
with open(data_path + "ranker_val.dill", "rb") as f:
    ranker_val = dill.load(f)

# ranker_val = add_users_features(ranker_val, df_users)
ranker_val = add_items_features(ranker_val, df_items)

# Save
with open(data_path + "ranker_val_final.dill", "wb") as f:
    dill.dump(ranker_val, f)

In [ ]:
# Загрузим таблицу ranker_test
with open(data_path + "ranker_test.dill", "rb") as f:
    ranker_test = dill.load(f)

# ranker_test = add_users_features(ranker_test, df_users)
ranker_test = add_items_features(ranker_test, df_items)

# Save
with open(data_path + "ranker_test_final.dill", "wb") as f:
    dill.dump(ranker_test, f)

## Добавим таргет

In [ ]:
def add_target(df: pd.DataFrame) -> pd.DataFrame:

    df["target"] = np.where(df["ui_inter"] > 1, 2, 1)
    df["target"] = np.where(df["ui_inter"] > 2, 4, df["target"])
    df["target"] = np.where(df["ui_inter"] > 4, 8, df["target"])
    df["target"] = np.where(df["ui_inter"] > 6, 10, df["target"])
    df["target"] = df["target"].astype(int)

    return df

In [ ]:
# Загрузим таблицу ranker_train
with open(data_path + "ranker_train_final.dill", "rb") as f:
    ranker_train = dill.load(f)

# Загрузим таблицу ranker_val
with open(data_path + "ranker_val_final.dill", "rb") as f:
    ranker_val = dill.load(f)

# Загрузим таблицу ranker_test
with open(data_path + "ranker_test_final.dill", "rb") as f:
    ranker_test = dill.load(f)

In [ ]:
ranker_train = add_target(ranker_train)
ranker_val = add_target(ranker_val)
ranker_test = add_target(ranker_test)

In [ ]:
# Save 
with open(data_path + "ranker_train_final.dill", "wb") as f:
    dill.dump(ranker_train, f)

# Save
with open(data_path + "ranker_val_final.dill", "wb") as f:
    dill.dump(ranker_val, f)

# Save
with open(data_path + "ranker_test_final.dill", "wb") as f:
    dill.dump(ranker_test, f)

## TRAIN MODEL

In [ ]:
# Загрузим таблицу ranker_train
with open(data_path + "ranker_train_final.dill", "rb") as f:
    ranker_train = dill.load(f)
    
# Загрузим таблицу ranker_val
with open(data_path + "ranker_val_final.dill", "rb") as f:
    ranker_val = dill.load(f)

### Выбираем колонки на которых будет обучаться ранкер

In [ ]:
ranker_train.columns

In [ ]:
# Убираем айдишники
# (данные на которых обучались предыдущие модели уже убрали)
# Так же решил убрать weight и cum_weight, так как 
# target определенно зависит от ui_inter, а weight и cum_weight 
# выводились через ui_inter
cols = [
    "cos_score",
    "cos_rank",
    "bm25_score",
    "bm25_rank",
    "tfidf_score",
    "tfidf_rank",
    "lfm_score",
    "lfm_rank",
    "user_hist",
    "user_avg_pop",
    "user_last_pop",
    "title_len",
    "descr_len",
    "title_word_len",
    "descr_word_len",
    "txt_emb_pca_0",
    "txt_emb_pca_1",
    "txt_emb_pca_2",
    "txt_emb_pca_3",
    "txt_emb_pca_4",
    "txt_emb_pca_5",
    "txt_emb_pca_6",
    "txt_emb_pca_7",
    "txt_emb_pca_8",
    "txt_emb_pca_9",
    "brand",
    "color",
    "closure",
    "country",
    "cut",
    "height",
    "length",
    "material",
    "model",
    "neckline",
    "pattern",
    "pocket",
    "purpose",
    "sleeve",
    "img_pca_0",
    "img_pca_1",
    "img_pca_2",
    "img_pca_3",
    "img_pca_4",
    "img_pca_5",
    "img_pca_6",
    "img_pca_7",
    "img_pca_8",
    "img_pca_9",
    "item_pop",
    "item_avg_hist",
]
# Из cols следующие фитчи категориальные
cat_cols = [
    "brand",
    "color",
    "closure",
    "country",
    "cut",
    "height",
    "length",
    "material",
    "model",
    "neckline",
    "pattern",
    "pocket",
    "purpose",
    "sleeve",
]

### Группировка для LightGBM

In [ ]:
def get_group(df: pd.DataFrame) -> np.ndarray:
    return np.array(
        df[["user_id", "item_id"]].groupby(by=["user_id"]).count()["item_id"]
    )

### Параметры ранкера и обучение

In [ ]:
early_stopping_rounds = 32 # число итераций, в течение которых нет улучшения метрик
params = {
    "objective": "lambdarank",  # lambdarank, оптимизирующий ndcg
    "n_estimators": 1000,  
    "max_depth": 4,  
    "num_leaves": 10, 
    "min_child_samples": 100,  
    "learning_rate": 0.03, 
    "reg_lambda": 1, 
    "colsample_bytree": 0.9, 
    "early_stopping_rounds": early_stopping_rounds,  
    "verbose": early_stopping_rounds // 2,  # период вывода метрик
    "random_state": RANDOM_STATE,
}
fit_params = {
    "X": ranker_train[cols],
    "y": ranker_train["target"],
    "group": get_group(ranker_train),
    "eval_set": [(ranker_val[cols], ranker_val["target"])],
    "eval_group": [get_group(ranker_val)],
    "eval_metric": "ndcg",
    "eval_at": (3, 5, 10),
    "categorical_feature": cat_cols,
    "feature_name": cols,
}

listwise_model = LGBMRanker(**params)
listwise_model.fit(**fit_params)


## TEST RANKER

In [ ]:
# Загрузим таблицу ranker_test
with open(data_path + "ranker_test_final.dill", "rb") as f:
    ranker_test = dill.load(f)

## SHAP plots

In [ ]:
explainer = shap.Explainer(listwise_model)
shap_values = explainer(ranker_test[cols].iloc[:10_000])

### WaterFall plot

In [ ]:
shap.plots.waterfall(shap_values[0], max_display=len(cols))
shap.plots.waterfall(shap_values[1], max_display=len(cols))
shap.plots.waterfall(shap_values[200], max_display=len(cols))

### beeswarm plot

In [ ]:
# summarize the effects of all the features
shap.plots.beeswarm(shap_values, max_display=len(cols))

### bar plot

In [ ]:
# mean shap-values
shap.plots.bar(shap_values, max_display=len(cols))

## Feature importance plot

In [ ]:
sorted_idx = np.argsort(listwise_model.feature_importances_)
fig = plt.figure(figsize=(10, 10))
plt.barh(range(len(sorted_idx)), listwise_model.feature_importances_[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), np.array(cols)[sorted_idx])
plt.title('Ranker Feature Importance')
plt.show()

# Выдаем рекомендации


In [ ]:
y_pred: np.ndarray = listwise_model.predict(ranker_test[cols])

In [ ]:
def add_score_and_rank(
    df: pd.DataFrame, y_pred_scores: np.ndarray, name: str
) -> pd.DataFrame:
    
    # Добавляем скор модели второго уровня
    df[f"{name}_score"] = y_pred_scores
    # Добавляем ранг модели второго уровня
    df.sort_values(
        by=["user_id", f"{name}_score"],
        ascending=[True, False],
        inplace=True,
    )
    df[f"{name}_rank"] = df.groupby("user_id").cumcount() + 1

    # Исключаем айтемы, которые не были предсказаны на первом уровне
    mask = (
        (df["cos_rank"] < 15)
        | (df["bm25_rank"] < 15)
        | (df["lfm_rank"] < 15)
        | (df["tfidf_rank"] < 15)
    ).to_numpy()

    # Добавляем общий скор двухэтапной модели
    eps: float = 0.001
    min_score: float = min(y_pred_scores) - eps
    df[f"{name}_hybrid_score"] = df[f"{name}_score"] * mask
    df[f"{name}_hybrid_score"].replace(
        0,
        min_score,
        inplace=True,
    )

    # Добавляем общий ранг двухэтапной модели
    df[f"{name}_hybrid_rank"] = df[f"{name}_rank"] * mask
    max_rank: int = 101
    df[f"{name}_hybrid_rank"].replace(
        0,
        max_rank,
        inplace=True,
    )

    return df

In [ ]:
ranker_test = add_score_and_rank(ranker_test, y_pred, "listwise")
ranker_test.head(3)

## Считаем метрики

In [ ]:
predictions = (
    ranker_test[ranker_test["listwise_hybrid_rank"] <= 10][["user_id", "item_id"]]
    .groupby(by="user_id")["item_id"]
    .apply(list)
    .reset_index()
    .rename(columns={"item_id": "listwise_hybrid_rank_recs"})
)
predictions

In [ ]:
# Загружаем таблицу test_df
with open(data_path + "test_df.dill", "rb") as f:
    test_df = dill.load(f)
    
test_df

In [ ]:
test_df = pd.merge(
    test_df[test_df["user_id"].isin(predictions["user_id"].unique())],
    predictions,
    how="left",
    on="user_id",
)
test_df

In [ ]:
RecommenderMetrics.evaluate_recommender(
    test_df, model_preds_col="listwise_hybrid_rank_recs"
)

# Выводы

### Следующими действиями по оптимизации работы модели будут: 
- подбор параметров ранкера
- тест большего числа данных для обучения ранкера
- интерпретация получаемых результатов

### План дальнейших работ: 
- переобучить модели первого уровня на дополнительных данных (за период ранкера)
- выдать рекомендации моделью второго уровня всем пользователям, присутствующим до начала test выборки и посчитать метрики для таргетированных рекомендаций
- выдать рекомендации оставшимся пользователям (появившимся во время test) (выдавать рекомендации будем либо PopularModel, либо BanditRecommender)